In [1]:
import duckdb
con = duckdb.connect(database=':memory:', read_only=False)

In [2]:
# 1. '판매 기록 (Sales)' 테이블 생성

con.execute("""
CREATE TABLE Sales (
    판매ID INTEGER PRIMARY KEY,
    상품코드 VARCHAR(10),
    판매량 INTEGER
);
""")


In [3]:
# 2. '판매 기록 (Sales)' 데이터 삽입

con.execute("""
INSERT INTO Sales (판매ID, 상품코드, 판매량) VALUES
(1, 'A1', 100),
(2, 'B2', 50),
(3, 'A1', 120),
(4, 'C3', 30),
(5, 'D4', 80);  -- D4는 상품 정보 테이블에 없는 코드
""")

In [4]:
# 3. 데이터 확인

print("--- [테이블 1] Sales 데이터 확인 ---")
print(con.execute("SELECT * FROM Sales;").fetchdf())


--- [테이블 1] Sales 데이터 확인 ---
   판매ID 상품코드  판매량
0     1   A1  100
1     2   B2   50
2     3   A1  120
3     4   C3   30
4     5   D4   80


In [5]:
# 4. '상품 정보 (Products)' 테이블 생성

con.execute("""
CREATE TABLE Products (
    코드 VARCHAR(10) PRIMARY KEY,
    상품명 VARCHAR(50),
    가격 INTEGER
);
""")

In [6]:
# 5. '상품 정보 (Products)' 데이터 삽입

con.execute("""
INSERT INTO Products (코드, 상품명, 가격) VALUES
('A1', '커피', 4500),
('B2', '케이크', 6000),
('C3', '샌드위치', 5000),
('E5', '샐러드', 8000);  -- E5는 판매 기록 테이블에 없는 코드
""")

In [7]:
# 6. 데이터 확인

print("\n--- [테이블 2] Products 데이터 확인 ---")
print(con.execute("SELECT * FROM Products;").fetchdf())


--- [테이블 2] Products 데이터 확인 ---
   코드   상품명    가격
0  A1    커피  4500
1  B2   케이크  6000
2  C3  샌드위치  5000
3  E5   샐러드  8000


In [ ]:
# INNER JOIN (내부 조인)은
# 두 테이블 간의 공통된 값(키)이 있는 행만을 결합하여 하나의 결과를 만듬

In [8]:
# 판매ID(Sales), 상품코드(Sales), 판매량(Sales)        코드(Products), 상품명(Products), 가격(Products)
# 1,            A1,             100,                A1,             커피,            4500
# 2,            B2,             50,                 B2,             케이크,          6000
# 3,            A1,             120,                A1,             커피,            4500
# 4,            C3,             30,                 C3,             샌드위치,         5000

In [12]:
inner_join_query = """
SELECT * FROM Sales AS S INNER JOIN Products AS P ON S.상품코드 = P.코드 ORDER BY S.판매ID;
"""

In [ ]:
# SELECT * FROM Sales AS S INNER JOIN Products AS P ON S.상품코드 = P.코드 ORDER BY S.판매ID;
# -----1-------
#               ------------------2----------------
#                                                   ----------3---------
#                                                                        --------4--------

# Sales AS S: Sales 테이블을 간단히 S라고 부르겠다는 의미(별칭을 만든 것)
# Products AS P: Products 테이블을 간단히 P라고 부르겠다는 의미(별칭을 만든 것)

# 1. SELECT를 실행(테이블에서 내용을 가져와서 보여달라)
# 2. 테이블에 해당, AS S, AS P를 없애고 보면 Sales INNER JOIN Products의 형태임
# 3. ON=은 INNER JOIN의 기준 즉, 두 테이블 간의 공통된 값이 무엇인지 알려주는 곳


# 두 테이블 간의 공통된 값은
# Sales 테이블의 상품코드 필드의 값과
# Products 테이블의 코드 필드의 값이 동일함
# 따라서 두 테이블 간의 공통된 값이 무엇인지에 해당하는 부분은 Sales의 상품코드와 Products의 코드임
# 그런데 Sales는 간단히 S라고 한다고 했고, Products는 간단히 P라고 한다고 했음
# Sales 테이블의 상품코드는 Sales.상품코드 >>> S.상품코드
# Products 테이블의 코드는 Products.코드 >>> P.코드


# ORDER BY S.판매ID는
# Sales 테이블의 판매ID 오름차순으로 정렬해서 SELECT 하라는 의미

In [ ]:

print("--- INNER JOIN 쿼리 실행 결과 ---")
result_df = con.execute(inner_join_query).fetchdf()
print(result_df)



--- INNER JOIN 쿼리 실행 결과 ---
   판매ID 상품코드  판매량  코드   상품명    가격
0     1   A1  100  A1    커피  4500
1     2   B2   50  B2   케이크  6000
2     3   A1  120  A1    커피  4500
3     4   C3   30  C3  샌드위치  5000


In [11]:
inner_join_query = """
SELECT
    S.판매ID,
    P.상품명,
    S.판매량,
    P.가격
FROM Sales AS S
INNER JOIN Products AS P
  ON S.상품코드 = P.코드
ORDER BY S.판매ID;
"""

In [ ]:

print("--- INNER JOIN 쿼리 실행 결과 ---")
result_df = con.execute(inner_join_query).fetchdf()
print(result_df)



--- INNER JOIN 쿼리 실행 결과 ---
   판매ID 상품코드  판매량  코드   상품명    가격
0     1   A1  100  A1    커피  4500
1     2   B2   50  B2   케이크  6000
2     3   A1  120  A1    커피  4500
3     4   C3   30  C3  샌드위치  5000


In [ ]:
con.close()